This notebook shows an example of importing and scheduling another notebook in ModelOps. The notebook that's being scheduled is available [in artifacts directory](./artifacts/BYO_Notebook.ipynb).

# Initialize a client

First, we need to initialize a client. It will pickup ModelOps instance configuration automatically from `~/.tmo/` directory. Alternatively, you could provide connection parameters to `TmoClient` constructor.

In [ ]:
from tmo import TmoClient

client = TmoClient()

Let's add a couple of helper functions that will help us to format some API outputs, and let's see if we have a project configured.

In [ ]:
import pandas as pd

pd.set_option("max_colwidth", 140)


def dict_to_frame(d, columns=None):
    if columns:
        return pd.DataFrame(
            [[k, v] for (k, v) in d.items() if k in columns],
            columns=["attribute", "value"],
        )
    else:
        return pd.DataFrame(list(d.items()), columns=["attribute", "value"])


def list_to_frame(d, columns):
    return pd.DataFrame(
        [[l.get(column) for column in columns] for l in d], columns=columns
    )


list_to_frame(
    list(client.projects()),
    ["id", "name", "description", "groupId", "gitRepositoryUrl", "createdAt"],
)

Next, we'll configure a current project for our client instance. It will allow us to skip project ID for all other calls we make.

In [ ]:
client.set_project_id("70d4659b-92a2-4723-841a-9ba5629b5f27")

Now we can ask ModelOps to describe this project.

In [ ]:
client.describe_current_project()

# Define BYOM model and import model version (the notebook itself)

First, we should define what we want to import:
1. The notebook itself
2. Optionally, `requirements.txt` file for any additional Python requirements
3. Any additional artifacts

In [ ]:
files = ["./artifacts/BYO_Notebook.ipynb", "./artifacts/requirements.txt"]
language = "Jupyter"

Next, let's define a model in ModelOps, under which we'll register this notebook as a new model version. Alternatively, you could use an existing model, make sure its ID is in `model` variable.

In [ ]:
import uuid

import_id = str(uuid.uuid4())

model_dict = {
    "name": f"{language}_Demo_{uuid.uuid4().clock_seq}",
    "description": f"{language} model defined from Python SDK",
    "language": language,
}
model_response = client.models().save(model_dict)
model = model_response["id"]

Next, we upload desired artifacts. Notice that we should preserve `import_id` for additional imports and to instruct ModelOps to registed imported artifacts as BYOM model version.

In [ ]:
client.trained_model_artefacts().upload_artefacts(import_id, artefacts=files)

Now, we can register this imported artifact as a BYOM model version. `import_request` here is very simple:
- `artefactImportId` is that `import_id` we've used before to upload artifacts
- `externalId` is any non-empty string that you could use to track this particular artifact (e.g. Notebook identifier in a different system)

In [ ]:
import_request = {"artefactImportId": import_id, "externalId": str(uuid.uuid4())}
import_job = client.models().import_byom(model, import_request)
imported_model_id = import_job["metadata"]["trainedModel"]["id"]
client.jobs().wait(import_job["id"])
print("Model imported")

# Approve imported model version (notebook) for production deployment

The next step is simple, but important. We have to approve the model version in order to allow us (and others) to deploy it. We need just a model version ID, and we are free to provide any comment for the approval.

In [ ]:
dict_to_frame(client.trained_models().approve(imported_model_id, comments="LGTM"))

# Schedule a notebook for single execution

Once the model version is approved, we can schedule it for regular (or singular) execution. First, we need a `dataset_template` identifier, that will be provided as metadata to our notebook.

In [ ]:
dataset_template = client.dataset_templates().find_by_name_like("PIMA")["_embedded"][
    "datasetTemplates"
][0]
dict_to_frame(dataset_template, columns=["id", "name", "description"])

Next, we define a deployment request and use it to deploy the model version. This deployment doesn't have a schedule defined (see `cron` value), and we'll trigger scoring jobs for it manually.

In [ ]:
deploy_request = {
    "engineType": "DOCKER_BATCH",
    "engineTypeConfig": {
        "dockerImage": (
            "artifacts.td.teradata.com/tdproduct-docker-snapshot/avmo/vmo-python-base:3.11.4"
        ),
        "engine": "jupyter",
        "resources": {
            "memory": "2g",
            "cpu": "1",
        },
    },
    "language": "jupyter",
    "datasetConnectionId": client.get_default_connection_id(),
    "byomModelLocation": {"database": "trng_modelops", "table": "vmo_byom_models"},
    "datasetTemplateId": dataset_template["id"],
    "cron": "None",
    "publishOnly": "false",
    "customProperties": {},
}

deploy_job = client.trained_models().deploy(imported_model_id, deploy_request)

client.jobs().wait(deploy_job["id"])
deployment = client.deployments().find_by_deployment_job_id(deploy_job["id"])
print("Model deployed")

# Trigger the prediction job
Now we can trigger prediction jobs manually. Please note `args` dictionary, it could be used to pass any scoring requests.

In [ ]:
scoring_request = {
    "args": {"USER_VARIABLE ": "scoring_job"},
    "datasetConnectionId": "6b07ae1e-0d79-46a7-8f9b-2f05ca957cdc",
    "automationOverrides": {
        "resources": {"memory": "1g", "cpu": "1"},
        "dockerImage": (
            "artifacts.td.teradata.com/tdproduct-docker-snapshot/avmo/vmo-python-base:3.11.4"
        ),
    },
}
client.deployments().run_scoring(deployment["id"], scoring_request)
print("Scoring job submitted")

# Monitor the prediction job

Once the model version is deployed, we should start looking for prediction jobs initiated by this deployment:

In [ ]:
jobs = client.jobs().find_by_deployment_id(deployment["id"], "expandJob")["_embedded"][
    "jobs"
]
list_to_frame(jobs, ["id", "type", "status", "createdAt", "updatedAt"])

Looks like there's at least a single job, let's monitor it:

In [ ]:
if len(jobs) == 1:
    client.jobs().wait(jobs[0]["id"])
    print("Job completed")
elif len(jobs) == 0:
    print("No jobs found")
else:
    print("Multiple jobs found")